In [1]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import random

import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [32]:
output_dir = Path("/Volumes/Lixi_Liu/panels_results_550k/plots/tea")
output_dir.mkdir(parents=True, exist_ok=True)
print(f"output dir: {output_dir}")

output dir: /Volumes/Lixi_Liu/panels_results_550k/plots/tea


In [2]:
panel_screen_file = Path("/Volumes/Lixi_Liu/panels_results_550k/plots/tea/panel_screens_nec_2023_L2_for_analysis.parquet")
df = pd.read_parquet(panel_screen_file)
df

predicted_panel_amp_bin  \
nec_version ev_level case nec_section building_id                           
2023        L2       1    83          2                              <100   
                          87          2                              <100   
                          83          3                               100   
                          87          3                               100   
                          83          4                               200   
...                                                                   ...   
                     19   87          549997                          100   
                          83          549998                         <100   
                          87          549998                         <100   
                          83          549999                          200   
                          87          549999                          200   

                                                   capacity_constraint  \
nec_version ev_level case nec_section building_id                        
2023        L2       1    83          2                           True   
                          87          2                           True   
                          83          3                          False   
                          87          3                          False   
                          83          4                          False   
...                                                                ...   
                     19   87          549997                      True   
                          83          549998                     False   
                          87          549998                     False   
                          83          549999                     False   
                          87          549999                     False   

                                                   space_constraint  \
nec_version ev_level case nec_section building_id                     
2023        L2       1    83          2                       False   
                          87          2                       False   
                          83          3                        True   
                          87          3                        True   
                          83          4                       False   
...                                                             ...   
                     19   87          549997                  False   
                          83          549998                  False   
                          87          549998                  False   
                          83          549999                  False   
                          87          549999                  False   

                                                  constraint_type  \
nec_version ev_level case nec_section building_id                   
2023        L2       1    83          2             capacity_only   
                          87          2             capacity_only   
                          83          3                space_only   
                          87          3                space_only   
                          83          4             no_constraint   
...                                                           ...   
                     19   87          549997        capacity_only   
                          83          549998        no_constraint   
                          87          549998        no_constraint   
                          83          549999        no_constraint   
                          87          549999        no_constraint   

                                                   has_constraint  \
nec_version ev_level case nec_section building_id                   
2023        L2       1    83          2                      True   
                          87          2                 

### BAU L2 using 220.83

In [71]:
# downselect to BAU with L2 charging, SF only, 220.83
bau = df.loc[(2023, "L2", 2, 83)]
cond = bau["build_existing_model.geometry_building_type_recs"].isin(["Single-Family Attached", "Single-Family Detached"])
bau = bau.loc[cond]

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_23435/2649198169.py:2: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [72]:
# create map on breaker space constraint
panel_label1 = "% dwelling units"
title = "% Single-family homes with breaker space constraint"
baucond = bau.loc[bau["constraint_type"].isin(["space_only", "capacity_and_space"])]
data1 = (baucond.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count() / \
        bau.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count()*100).rename(panel_label1)
data1.index.name = "State"
data1 = data1.reset_index()

# plot
fig = px.choropleth(data1, locations='State', color=panel_label1, title=title,
                    locationmode="USA-states", color_continuous_scale="tempo", scope="usa")
# remove DC and RI
data = data1.loc[~data1["State"].isin(["DC", "DE", "NH", "RI"])]
fig.add_scattergeo(locations=data['State'], text=data[panel_label1].astype(float).round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   showlegend=False,
                  )
# DC
fig.add_scattergeo(
    lat = [38.889805, 36], 
    lon = [-77.009056, -73],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# DE
fig.add_scattergeo(
    lat = [39, 37], 
    lon = [-75.5, -72],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# NH
fig.add_scattergeo(
    lat = [44, 42],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# RI
fig.add_scattergeo(
    lat = [41.742325, 39],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
data = data1.loc[data1["State"].isin(["DC", "DE" , "NH", "RI"])].copy()
data["label"] = data["State"].astype(str)+", "+data[panel_label1].astype(float).round(0).astype(int).astype(str)
fig.add_scattergeo(lat=[36, 37, 42, 39], lon=[-73, -72, -68, -68], text=data["label"], 
                   mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   textposition="middle right",
                   showlegend=False,
                  )
fig.show()
fig_file = f"map__{panel_label1}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
pio.write_image(fig, fig_file, scale=6, width=800, height=500)

In [73]:
# create map on capacity constraint
panel_label1 = "% dwelling units"
title = "% Single-family homes with capacity constraint (220.83)"
baucond = bau.loc[bau["constraint_type"].isin(["capacity_only", "capacity_and_space"])]
data1 = (baucond.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count() / \
        bau.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count()*100).rename(panel_label1)
data1.index.name = "State"
data1 = data1.reset_index()

# plot
fig = px.choropleth(data1, locations='State', color=panel_label1, title=title,
                    locationmode="USA-states", color_continuous_scale="amp", scope="usa")
# remove DC and RI
data = data1.loc[~data1["State"].isin(["DC", "DE", "NH", "RI"])]
fig.add_scattergeo(locations=data['State'], text=data[panel_label1].astype(float).round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   showlegend=False,
                  )
# DC
fig.add_scattergeo(
    lat = [38.889805, 36], 
    lon = [-77.009056, -73],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# DE
fig.add_scattergeo(
    lat = [39, 37], 
    lon = [-75.5, -72],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# NH
fig.add_scattergeo(
    lat = [44, 42],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# RI
fig.add_scattergeo(
    lat = [41.742325, 39],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
data = data1.loc[data1["State"].isin(["DC", "DE" , "NH", "RI"])].copy()
data["label"] = data["State"].astype(str)+", "+data[panel_label1].astype(float).round(0).astype(int).astype(str)
fig.add_scattergeo(lat=[36, 37, 42, 39], lon=[-73, -72, -68, -68], text=data["label"], 
                   mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   textposition="middle right",
                   showlegend=False,
                  )
fig.show()
fig_file = f"map__{panel_label1}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
pio.write_image(fig, fig_file, scale=6, width=800, height=500)

In [74]:
# create map on capacity constraint
panel_label1 = "% dwelling units"
title = "% Single-family homes with capacity or space constraint (220.83)"
baucond = bau.loc[~bau["constraint_type"].isin(["no_constraint"])]
data1 = (baucond.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count() / \
        bau.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count()*100).rename(panel_label1)
data1.index.name = "State"
data1 = data1.reset_index()

# plot
fig = px.choropleth(data1, locations='State', color=panel_label1, title=title,
                    locationmode="USA-states", color_continuous_scale="deep", scope="usa")
# remove DC and RI
data = data1.loc[~data1["State"].isin(["DC", "DE", "NH", "RI"])]
fig.add_scattergeo(locations=data['State'], text=data[panel_label1].astype(float).round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   showlegend=False,
                  )
# DC
fig.add_scattergeo(
    lat = [38.889805, 36], 
    lon = [-77.009056, -73],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# DE
fig.add_scattergeo(
    lat = [39, 37], 
    lon = [-75.5, -72],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# NH
fig.add_scattergeo(
    lat = [44, 42],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# RI
fig.add_scattergeo(
    lat = [41.742325, 39],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
data = data1.loc[data1["State"].isin(["DC", "DE" , "NH", "RI"])].copy()
data["label"] = data["State"].astype(str)+", "+data[panel_label1].astype(float).round(0).astype(int).astype(str)
fig.add_scattergeo(lat=[36, 37, 42, 39], lon=[-73, -72, -68, -68], text=data["label"], 
                   mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   textposition="middle right",
                   showlegend=False,
                  )
fig.show()
fig_file = f"map__{panel_label1}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
pio.write_image(fig, fig_file, scale=6, width=800, height=500)

### BAU L2 using 220.87

In [75]:
# downselect to BAU with L2 charging, SF only, 220.87
bau2 = df.loc[(2023, "L2", 2, 87)]
cond = bau2["build_existing_model.geometry_building_type_recs"].isin(["Single-Family Attached", "Single-Family Detached"])
bau2 = bau2.loc[cond]

# create map
panel_label1 = "% dwelling units"
title = "% Single-family homes with capacity constraint (220.87)"
baucond2 = bau2.loc[bau2["constraint_type"].isin(["capacity_only", "capacity_and_space"])]
data1 = (baucond2.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count() / \
        bau2.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count()*100).rename(panel_label1)
data1.index.name = "State"
data1 = data1.reset_index()

# plot
fig = px.choropleth(data1, locations='State', color=panel_label1, title=title,
                    locationmode="USA-states", color_continuous_scale="amp", scope="usa")
# remove DC and RI
data = data1.loc[~data1["State"].isin(["DC", "DE", "NH", "RI"])]
fig.add_scattergeo(locations=data['State'], text=data[panel_label1].astype(float).round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   showlegend=False,
                  )
# DC
fig.add_scattergeo(
    lat = [38.889805, 36], 
    lon = [-77.009056, -73],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# DE
fig.add_scattergeo(
    lat = [39, 37], 
    lon = [-75.5, -72],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# NH
fig.add_scattergeo(
    lat = [44, 42],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# RI
fig.add_scattergeo(
    lat = [41.742325, 39],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
data = data1.loc[data1["State"].isin(["DC", "DE" , "NH", "RI"])].copy()
data["label"] = data["State"].astype(str)+", "+data[panel_label1].astype(float).round(0).astype(int).astype(str)
fig.add_scattergeo(lat=[36, 37, 42, 39], lon=[-73, -72, -68, -68], text=data["label"], 
                   mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   textposition="middle right",
                   showlegend=False,
                  )
fig.show()
fig_file = f"map__{panel_label1}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
pio.write_image(fig, fig_file, scale=6, width=800, height=500)

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_23435/540698584.py:2: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [76]:
# downselect to BAU with L2 charging, SF only, 220.87
bau2 = df.loc[(2023, "L2", 2, 87)]
cond = bau2["build_existing_model.geometry_building_type_recs"].isin(["Single-Family Attached", "Single-Family Detached"])
bau2 = bau2.loc[cond]

# create map
panel_label1 = "% dwelling units"
title = "% Single-family homes with capacity or space constraint (220.87)"
baucond2 = bau2.loc[~bau2["constraint_type"].isin(["no_constraint"])]
data1 = (baucond2.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count() / \
        bau2.groupby(["build_existing_model.state"])["predicted_panel_amp_bin"].count()*100).rename(panel_label1)
data1.index.name = "State"
data1 = data1.reset_index()

# plot
fig = px.choropleth(data1, locations='State', color=panel_label1, title=title,
                    locationmode="USA-states", color_continuous_scale="deep", scope="usa")
# remove DC and RI
data = data1.loc[~data1["State"].isin(["DC", "DE", "NH", "RI"])]
fig.add_scattergeo(locations=data['State'], text=data[panel_label1].astype(float).round(0), 
                   locationmode="USA-states", mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   showlegend=False,
                  )
# DC
fig.add_scattergeo(
    lat = [38.889805, 36], 
    lon = [-77.009056, -73],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# DE
fig.add_scattergeo(
    lat = [39, 37], 
    lon = [-75.5, -72],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# NH
fig.add_scattergeo(
    lat = [44, 42],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
# RI
fig.add_scattergeo(
    lat = [41.742325, 39],
    lon = [-71.5, -68],
    mode = 'lines',
    line = dict(width = 1, color = 'grey'),
    showlegend=False,
)
data = data1.loc[data1["State"].isin(["DC", "DE" , "NH", "RI"])].copy()
data["label"] = data["State"].astype(str)+", "+data[panel_label1].astype(float).round(0).astype(int).astype(str)
fig.add_scattergeo(lat=[36, 37, 42, 39], lon=[-73, -72, -68, -68], text=data["label"], 
                   mode='markers+text', opacity=1,
                   textfont=dict(color="black", size=10),
                   marker=dict(color="white", size=15, opacity=0.6),
                   textposition="middle right",
                   showlegend=False,
                  )
fig.show()
fig_file = f"map__{panel_label1}_{title}.png"
if output_dir is not None:
    fig_file = output_dir / fig_file
pio.write_image(fig, fig_file, scale=6, width=800, height=500)

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_23435/280859180.py:2: PerformanceWarning:

indexing past lexsort depth may impact performance.

